# Erwan's amateur accounting stuff

In [5]:
import pandas as pd

private_dir_open = False

file = '/home/erwan/private/finances/compta/BOI.ods'
sheet_name = 'Data'

if private_dir_open:
    data = pd.read_excel(file, engine="odf", sheet_name = sheet_name)
    print(data.tail(3))



### Tests bank statements

It should be possible to specify the format (even several possible formats) explicitly. Parameters:

- encoding
- separator (should be TAB if \t)
- table_first_row 
- column no for: (column nos indexed from 1)
    - col_date
    - col_object
    - col_amount (possibly two cols: cred:deb)
    - col_balance (if no column, absolute cell?)
- decimal_point_char
- date_format_pos_day


NO (The system will use its best guess for anything not specified?)

The UI could propose the detected format config and store it directly in the output file.

## Amount columns

- If single amount column, must be the amount
- If 3 amount columns, then must be cred, deb, balance. check that cred - deb = balance
- If 2 amount columns: if prev balance + amount = current balance then amount, balance; otherwise cred, deb

## About date format

The date format cannot reliably be "guessed" from a date string in all cases, even though the `dateutil` package provides a function for that (see below): in case of an ambiguous date such as 02/03/2024, the function assumes the US format (or maybe uses the locale and by default US locale?).

My approach: assume the date is made of 8 digits 2+2+4 in any order, separated by predefined possible separators (e.g. `/` or `-`). The 4 digits is obviously the year. Between the two 2 digits positions, search across rows if any of the numbers is higher than 12, marking this position as day. If no such luck, raise a warning and assume European format.
 

In [1]:
import dateutil
from dateutil.parser import parse

mydate = parse('02/12/2024')
print(mydate)
mydate = parse('24-12-2024')
print(mydate)
mydate = parse('02/03/2024')
print(mydate)
mydate = parse('21/12/2024')
print(mydate)


2024-02-12 00:00:00
2024-12-24 00:00:00
2024-02-03 00:00:00
2024-12-21 00:00:00


In [30]:
from collections import Counter
import locale
from typing import List
import warnings

potential_date_separators = [ '/', '-' ]
potential_csv_separators = [ ',', '\t', ';' ]
potential_decimal_separators = [ '.', ',' ]


file1 = '0656824Y0321723904987513.tsv'

# data = pd.read_csv(file, 
#                     encoding_errors = 'replace',
#                     on_bad_lines='warn')

# returns (first, length) most likely row indexes of the table
def regular_len_pattern(len_array):
    curr_seq = [0, 1, len_array[0]]
    max_seq = {}
    for i in range(1, len(len_array)):
        if len_array[i-1] == len_array[i]:
            curr_seq[1] += 1
        else:
            if (max_seq.get(curr_seq[2]) is None or curr_seq[1]-curr_seq[0] > max_seq[curr_seq[2]][1]):
                max_seq[curr_seq[2]] = (curr_seq[0], curr_seq[1]-curr_seq[0])
            curr_seq = [i, i+1, len_array[i]]
    # last sequence
    if (max_seq.get(curr_seq[2]) is None or curr_seq[1]-curr_seq[0] > max_seq[curr_seq[2]][1]):
        max_seq[curr_seq[2]] = (curr_seq[0], curr_seq[1]-curr_seq[0])
    longest_n_col = None
    for n_col,(start, l) in max_seq.items():
        if n_col > 2 and l > 3:
            if longest_n_col is None or l > longest_n_col:
                longest_n_col = n_col
    if longest_n_col is None:
        return 0,0
    else:
        return max_seq[longest_n_col][0],max_seq[longest_n_col][1]

def guess_day_position(date_as_string, sep):
    is_day = [ len(item)==2 and item >12 for item in date_as_string.split(sep) ]
    if True in is_day:
        return is_day.index(True)
    else:
        return None


def guess_column_content(value: str):
    # we suppose that a date has exactly 8 digits, two separators
    # an amount/balance has digits with optionally a single comma separator, optionally a minus first char
    # an object is mostly text
    n_digits = [ c.isdigit() for c in value ]
    n_letters = [ c.isalpha() for c in value ]
    if n_letters == 0: 
        if n_digits == len(value):
            return 'amount', None
        seps = set([ c for c in value if not c.isdigit() ])
        if  n_digits + 1 == len(value) and next(iter(seps)) in potential_decimal_separators:
            return 'amount', next(iter(seps))
        if n_digits + 2 == len(value):
            if len(seps == 1) and next(iter(seps)) in potential_date_separators:
                return 'date', guess_day_position(value, next(iter(seps)))
            elif len(seps) == 2 and value[0] == '-':
                other_sep = None
                for sep in seps:
                    if sep != '-':
                        if sep in potential_decimal_separators:
                            other_sep = sep
                if other_sep is not None:
                    return 'amount', other_sep
    return 'text', None


def guess_single_column(table: List[List[str]], col_no: int):
    content_types = [ guess_column_content(row[col_no])[0] for row in table ]
    count_content_types = Counter(content_types)
    if None in count_content_types.keys():
        del count_content_types[None]
    if len(count_content_types) == 1:
        res = content_types[0]
        if res[0] == 'date':
            res = next(x for x in content_types if x[1] is not None)
        return res
    else:
        raise Exception(f'Could not determine the content type in column {col_no+1} in the bank statement file.')

def check_three_cols_with_balance(table: List[List[str]], cols_no: List[int]):
    credit_col = cols_no[0]
    debit_col = cols_no[1]
    balance_col = cols_no[2]
    for i in range(1,len(table[0])):
        prev_bal = float(table[i-1][balance_col])
        credit = float(table[i][credit_col])
        debit = - float(table[i][debit_col])
        cur_bal = float(table[i][balance_col])
        if prev_bal + credit - debit != cur_bal:
            print(f'DEBUG {prev_bal} + {credit} - {debit} != {cur_bal}')
            return False
    return True


def check_two_cols_with_balance(table: List[List[str]], cols_no: List[int]):
    amount_col = cols_no[0]
    balance_col = cols_no[1]
    for i in range(1,len(table[0])):
        prev_bal = float(table[i-1][balance_col])
        cur_amount = float(table[i][amount_col])
        cur_bal = float(table[i][balance_col])
        if prev_bal + cur_amount != cur_bal:
            print(f'DEBUG {prev_bal} + {cur_amount} != {cur_bal}')
            return False
    return True
    
def guess_columns(table: List[List[str]]):
    bank_format_conf = {}
    amount_cols = []
    for col_no in range(len(table[0])):
        col_type = guess_single_column(table, col_no)
        if col_type[0] == 'date':
            if bank_format_conf.get('col_date') is None:
                bank_format_conf['col_date'] = col_no+1
            else:
                raise Exception(f'Two date columns found in the bank statement file: {bank_format_conf['col_date']} and {col_no+1}')
            bank_format_conf['date_format_pos_day'] = col_type[1] +1
        elif col_type[0] == 'text':
            if bank_format_conf.get('col_object') is None:
                bank_format_conf['col_object'] = col_no+1
            else:
                raise Exception(f'Two text columns found in the bank statement file: {bank_format_conf['col_object']} and {col_no+1}')
        elif col_type[0] == 'amount':
            bank_format_conf['decimal_point_char'] = col_type[1]
            amount_cols.append(col_no)
    if len(amount_cols) == 1:
        bank_format_conf['col_amount'] = [amount_cols[0]+1]
        bank_format_conf['col_balance'] = 0
        warnings.warn('Warning: no balance column and no balance cell specified in the bank statement file, this is unsafe.')
    elif len(amount_cols) == 2:
        if check_two_cols_with_balance(table, amount_cols):
            bank_format_conf['col_amount'] = [amount_cols[0]+1]
            bank_format_conf['col_balance'] = amount_cols[1]+1
        else:
            bank_format_conf['col_amount'] = [amount_cols[0]+1,amount_cols[1]+1]
            bank_format_conf['col_balance'] = 0
    elif len(amount_cols) == 3:
        if check_three_cols_with_balance(table, amount_cols):
            bank_format_conf['col_amount'] = [amount_cols[0]+1,amount_cols[1]+1]
            bank_format_conf['col_balance'] = amount_cols[2]+1
        else:
            raise Exception('Three amount columns in the bank statement file but calculations failed.')
    else:
        raise Exception('More than three amount columns in the bank statement file, unable to parse.')


    return bank_format_conf

def decode_bank_file(filename: str):
    with open(filename, "r", errors='ignore') as f:
        data = [ line for line in f ]
    if len(data) > 0:
        data_tokens = { sep: [ line.split(sep) for line in data ] for sep in potential_csv_separators }
        len_line = { sep : [len(tokens)  for tokens in line_tokens ] for sep, line_tokens in data_tokens.items() }
        longest_seq0 = { sep: regular_len_pattern(a) for sep, a in len_line.items() }
        longest_seq = { sep: (start, len_seq) for sep, (start, len_seq) in longest_seq0.items() if len_seq > 0 }
        if len(longest_seq) == 0:
            raise Exception('Error: failed to detect the separator character.')
        if len(longest_seq) > 1:
            warnings.warn('Warning: more than one suitable separator detected, possible errors reading the data.')
        max_sep = None
        for sep, (start, len_seq) in longest_seq.items():
            if max_sep is None or len_seq > longest_seq[max_sep][1]:
                max_sep = sep
                max_start, max_len = start, len_seq
        # max_start+1 because the first row is supposed to be the header
        table = data_tokens[max_sep][max_start+1:max_start+max_len]
        # initialize conf here
        bank_format_conf = guess_columns(table)
        bank_format_conf['encoding'] = locale.getencoding()
        bank_format_conf['separator'] = max_sep 
        bank_format_conf['table_first_row'] = max_start
        

        
 
decode_bank_file(file1)



,
(0, 0)
	
(6, 19)
;
(0, 0)
